In [27]:
# por dia
import pandas as pd
import numpy as np
import os
import sys


## Events Separation

**Read a dataset and separate extreme precipitaiton events **


If the absolute value of the difference between current value of precipitation $[i]$ and the next value of precipitation $[i + 1]$ is greater than one and the accumulated precipitation from that moment to one hour later is greater than 15mm, then the value of precipitation $[i]$ is the start of the extreme precipitation
event.That is:

if $( |p[i] - p[i + 1]| > 1 $ and $\sum_{n=i}^{i+12} p[n] > 15 )$ then $p[i]$ is the start of the extreme precipitation
event.

Then it stores 12 values before $t_i$ and 11 eleven values after $t_i$. 

**In other words here, this code separtes events that lasted for and hour and the accumulated precicpitation was more than 15.0 mm**

In [28]:
def separate_event(EventsDataFrame,Dataset,directorio):
    # convert to numpy
    nump_array = Dataset.to_numpy()
    fechas=[]

    """print("\n\tjust checking dimention is well stablish\n")
    print("this is a row,nump_array[0]\n",nump_array[0],"\n\t therefore nump_array[0][j] is each element of that row")
    print("\n\nfecha",str(Dataset.index[0]))
    print("Valor (%)",nump_array[0][0])
    print("Valor (hPa), ",nump_array[0][1])
    print("Valor (W/m2),",nump_array[0][2])
    print("Valor (°C),",nump_array[0][3])
    print("Valor (mm),",nump_array[0][4])
    print('Monthly sin', nump_array[0][5])
    print('Monthly cos',nump_array[0][6])
    print('Bi-Monthly sin',nump_array[0][7])
    print('Bi-Monthly cos',nump_array[0][8])
    print("\t Wind X, ",nump_array[0][9])
    print("\t Wind Y, ",nump_array[0][10])"""

    #iii = input("con")

    # arrays that will be used to make .csv

    cont=0
    i =0
    while i < nump_array.shape[0]-12: # -12 in case last values cant continue
        condition1 = abs(nump_array[i][4] - nump_array[i+1][4])
        sum_acu    = 0
        threshold  = 15.0
        
        if (condition1>1):
            for p in range(i,i+12):                
                sum_acu = sum_acu + nump_array[p][4]

            if sum_acu > threshold:
                fecha = []
                Hr = []
                Ps = []
                Rs = []
                T  = []
                Pp = []
                Month_cos = []
                Month_sin = []
                BiMonth_cos = []
                BiMonth_sin = []                
                Wind_X = []
                Wind_Y = []
                labels = []
                #x=input("insede if")
                cont=cont+1
                for j in range(i-12,i+12):
                    fecha.append(Dataset.index[j])
                    Hr.append(str(nump_array[j][0]))
                    Ps.append(str(nump_array[j][1]))
                    Rs.append(str(nump_array[j][2]))
                    T.append(str(nump_array[j][3]))
                    Pp.append(str(nump_array[j][4]))
                    Month_sin.append(str(nump_array[j][5]))
                    Month_cos.append(str(nump_array[j][6]))
                    BiMonth_sin.append(str(nump_array[j][7]))
                    BiMonth_cos.append(str(nump_array[j][8]))
                    Wind_X.append(str(nump_array[j][9]))
                    Wind_Y.append(str(nump_array[j][10]))
                
                #make labels for this event (for three-class)
                for l in range(0,12):
                    labels.append('Extreme event is near')
                    #labels.append(-1)
                for l in range(13,25):
                    labels.append('Extreme event is happening')
                    #labels.append(0)

                #create data frame 
                col_names = ['Fecha', 'Value (%)', 'Value (hPa)','Value (W/m2)', 'Value (°C)', 'Value (mm)', 'Monthly sin','Monthly cos','Bi-monthly sin','Bi-monthly cos','Wind X','Wind Y','Label']

                dix = {'Fecha': fecha, 'Value (%)': Hr, 'Value (hPa)': Ps,'Value (W/m2)': Rs, 'Value (°C)': T, 'Value (mm)': Pp, 'Monthly sin': Month_sin, 'Monthly cos': Month_cos,'Bi-monthly sin':BiMonth_sin, 'Bi-monthly cos': BiMonth_cos, 'Wind X': Wind_X, 'Wind Y': Wind_Y, 'Label':labels}
                df = pd.DataFrame(dix, columns=col_names)                                                
                
                # save it as csv
                #print("\tcreating .csv of event in day", fecha[0])
                fecha_evento=fecha[0]
                fechas.append(fecha_evento[0:10])
                filename = directorio + fecha_evento[0:4]+'_'+fecha_evento[5:7] + '_'+fecha_evento[8:10]+'_'+str(cont)+'.csv'
                df.to_csv(filename, index=False)
                
                # make a dataframe only of extrm. precip. events
                EventsDataFrame=pd.concat([EventsDataFrame,df])                

                # skip used values in next iteration
                i=i+6 

        i=i+1 # analize next values

    #EventsDataFrame.to_csv(directorio+'EventsDataFrame.csv', index=False)
    print("total events", cont)

    return(EventsDataFrame)

## Start by extracting events from station Rumihurco

In [29]:
EventsDataFrame = pd.DataFrame()
path = "/home/juan/Desktop/TESIS/Codes/codesTesis/Paso1/Rumihurco.csv"
Dataset_Rumihurco = pd.read_csv(path,index_col=0)

directorio = '/home/juan/Desktop/TESIS/Codes/codesTesis/Paso2/Rumihurco/'
eventos_rumihurco = separate_event(EventsDataFrame,Dataset_Rumihurco,directorio)

indice = eventos_rumihurco.pop("Fecha")
eventos_rumihurco.index = indice
eventos_rumihurco

total events 9


,Value (%),Value (hPa),Value (W/m2),Value (°C),Value (mm),Monthly sin,Monthly cos,Bi-monthly sin,Bi-monthly cos,Wind X,Wind Y,Label
Fecha,,,,,,,,,,,,
2018/02/05 21:05:00,100.0,665.4,0.0,8.8,0.1,-0.6023956480982829,-0.798197646671706,-0.948208217289775,0.317649455633324,1.2944271909999199,0.940456403667957,Extreme event is near
2018/02/05 21:10:00,100.0,665.4,0.0,8.8,0.2,-0.602975954467736,-0.797759361169596,-0.9480926540084559,0.317994212864325,0.877624441943005,0.8183980320749978,Extreme event is near
2018/02/05 21:15:00,100.0,665.5,0.0,8.8,0.3,-0.603555941953558,-0.797320653772717,-0.9479769653775129,0.318338928052542,1.02308553935848,1.3576803670804,Extreme event is near
2018/02/05 21:20:00,100.0,665.5,0.0,8.8,0.1,-0.604135610249023,-0.7968815247130779,-0.947861151412241,0.318683601152398,1.1114533927344,1.15094368054184,Extreme event is near
2018/02/05 21:25:00,100.0,665.6,0.0,8.8,0.1,-0.604714959047574,-0.796441974222913,-0.9477452121279519,0.319028232118325,0.984088543485761,1.13206437033416,Extreme event is near
...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/06 18:50:00,92.0,664.1,0.0,6.8,0.2,-0.588961297699048,0.80816123998411,0.309708540418157,-0.950831541332141,-3.0760374270026203,-0.8820395386143969,Extreme event is happening
2019/11/06 18:55:00,91.0,664.2,0.0,7.0,0.0,-0.5883734305763471,0.8085893309905959,0.309362787847378,-0.9509440916769492,-2.9544232590366204,-0.520944533000791,Extreme event is happening
2019/11/06 19:00:00,92.0,664.3,0.0,6.8,0.0,-0.5877852522925561,0.809016994374887,0.309016994374996,-0.951056516295138,-0.8290375725550421,-0.5591929034707471,Extreme event is happening


## Repeat to extract events from station Rumipamba

In [30]:
EventsDataFrame = pd.DataFrame()
path = "/home/juan/Desktop/TESIS/Codes/codesTesis/Paso1/Rumipamba.csv"
Dataset_Rumipamba = pd.read_csv(path,index_col=0)

directorio = '/home/juan/Desktop/TESIS/Codes/codesTesis/Paso2/Rumipamba/'
eventos_rumipamba = separate_event(EventsDataFrame,Dataset_Rumipamba,directorio)

indice = eventos_rumipamba.pop("Fecha")
eventos_rumipamba.index = indice
eventos_rumipamba

total events 8


,Value (%),Value (hPa),Value (W/m2),Value (°C),Value (mm),Monthly sin,Monthly cos,Bi-monthly sin,Bi-monthly cos,Wind X,Wind Y,Label
Fecha,,,,,,,,,,,,
2019/10/24 12:25:00,70.0,683.6,223.0,13.0,0.0,0.263732757985087,-0.964595787034953,0.9911094256021759,0.133049263367084,-0.9325751537483651,0.7551844692598051,Extreme event is near
2019/10/24 12:30:00,67.0,683.5,277.0,13.1,0.0,0.263031214458007,-0.964787323828804,0.9911577381599771,0.132688877022899,-0.167757871041224,-0.24871127176651198,Extreme event is near
2019/10/24 12:35:00,69.0,683.5,294.0,13.4,0.0,0.262329531826958,-0.9649783503952041,0.991205919674415,0.132328473135595,-0.623704566931857,0.317793349817683,Extreme event is near
2019/10/24 12:40:00,64.0,683.4,236.0,13.6,0.0,0.261627710463024,-0.965168866633128,0.991253970139118,0.131968051752824,-0.958362612056751,0.7221780277824581,Extreme event is near
2019/10/24 12:45:00,63.0,683.2,210.0,13.5,0.0,0.260925750737363,-0.965358872441821,0.991301889547735,0.131607612922238,0.041868764994355004,-0.798903627803659,Extreme event is near
...,...,...,...,...,...,...,...,...,...,...,...,...
2020/03/10 14:05:00,97.0,684.9,72.0,9.4,1.4,0.367177748501522,0.930150794766823,-0.186881252715698,-0.9823825107275741,0.877624441943005,-0.8183980320749978,Extreme event is happening
2020/03/10 14:10:00,97.0,684.8,51.0,9.3,0.8,0.36785407609758297,0.9298835296414241,-0.187238444714989,-0.982314493846402,2.1563158771276902,-1.62490056251053,Extreme event is happening
2020/03/10 14:15:00,96.0,685.0,37.0,9.2,1.3,0.36853020915371704,0.9296157727475999,-0.18759561195880897,-0.982246347091095,2.10115455257798,-0.93549427907434,Extreme event is happening


## Make a training Dataset
1. Concat only events from both stations
2. Concat both stations
3. In order to get data with label 'Random Sample', get random samples from 2.
4. Check there are no repetead values
5. Make labels for the random samples
6. Concatenate Random Samples with Events DataFrame and make the final training dataset 

In [31]:
# 1. Concat only events from both stations

EventsDataFrame_C02_C04 = pd.concat([eventos_rumihurco,eventos_rumipamba])

# 2. Concat both stations
Dataset_C02_C04 = pd.concat([Dataset_Rumihurco,Dataset_Rumipamba])

In [32]:
Dataset_C02_C04

,Value (%),Value (hPa),Value (W/m2),Value (°C),Value (mm),Monthly sin,Monthly cos,Bi-monthly sin,Bi-monthly cos,Wind X,Wind Y
Fecha,,,,,,,,,,,
2017/12/04 13:45:00,84.0,663.2,139.0,10.2,0.0,0.045799,-0.998951,-0.999738,-0.022905,0.902723,1.197953
2017/12/04 13:50:00,83.0,663.2,104.0,10.4,0.0,0.045072,-0.998984,-0.999746,-0.022542,0.771784,1.514711
2017/12/04 13:55:00,86.0,663.2,69.0,10.3,0.0,0.044346,-0.999016,-0.999754,-0.022178,1.389572,1.295797
2017/12/04 14:00:00,86.0,663.2,103.0,10.2,0.0,0.043619,-0.999048,-0.999762,-0.021815,0.899863,0.015707
2017/12/04 14:05:00,84.0,663.1,177.0,10.2,0.0,0.042893,-0.999080,-0.999770,-0.021451,0.972522,1.007076
...,...,...,...,...,...,...,...,...,...,...,...
2020/04/14 11:15:00,84.0,687.0,409.0,11.7,0.0,0.985184,0.171499,0.643623,0.765343,-0.459627,0.385673
2020/04/14 11:20:00,87.0,686.9,854.0,12.6,0.0,0.985309,0.170783,0.643901,0.765109,0.262424,-0.301884
2020/04/14 11:25:00,79.0,686.9,250.0,13.0,0.0,0.985433,0.170066,0.644179,0.764875,-0.337146,0.834465


In [33]:
EventsDataFrame_C02_C04

,Value (%),Value (hPa),Value (W/m2),Value (°C),Value (mm),Monthly sin,Monthly cos,Bi-monthly sin,Bi-monthly cos,Wind X,Wind Y,Label
Fecha,,,,,,,,,,,,
2018/02/05 21:05:00,100.0,665.4,0.0,8.8,0.1,-0.6023956480982829,-0.798197646671706,-0.948208217289775,0.317649455633324,1.2944271909999199,0.940456403667957,Extreme event is near
2018/02/05 21:10:00,100.0,665.4,0.0,8.8,0.2,-0.602975954467736,-0.797759361169596,-0.9480926540084559,0.317994212864325,0.877624441943005,0.8183980320749978,Extreme event is near
2018/02/05 21:15:00,100.0,665.5,0.0,8.8,0.3,-0.603555941953558,-0.797320653772717,-0.9479769653775129,0.318338928052542,1.02308553935848,1.3576803670804,Extreme event is near
2018/02/05 21:20:00,100.0,665.5,0.0,8.8,0.1,-0.604135610249023,-0.7968815247130779,-0.947861151412241,0.318683601152398,1.1114533927344,1.15094368054184,Extreme event is near
2018/02/05 21:25:00,100.0,665.6,0.0,8.8,0.1,-0.604714959047574,-0.796441974222913,-0.9477452121279519,0.319028232118325,0.984088543485761,1.13206437033416,Extreme event is near
...,...,...,...,...,...,...,...,...,...,...,...,...
2020/03/10 14:05:00,97.0,684.9,72.0,9.4,1.4,0.367177748501522,0.930150794766823,-0.186881252715698,-0.9823825107275741,0.877624441943005,-0.8183980320749978,Extreme event is happening
2020/03/10 14:10:00,97.0,684.8,51.0,9.3,0.8,0.36785407609758297,0.9298835296414241,-0.187238444714989,-0.982314493846402,2.1563158771276902,-1.62490056251053,Extreme event is happening
2020/03/10 14:15:00,96.0,685.0,37.0,9.2,1.3,0.36853020915371704,0.9296157727475999,-0.18759561195880897,-0.982246347091095,2.10115455257798,-0.93549427907434,Extreme event is happening


In [34]:
len(EventsDataFrame_C02_C04)
n_each_labels = len(EventsDataFrame_C02_C04)/2
n_random_samples = 1.5*n_each_labels
n_random_samples

306.0

In [35]:
# 3. In order to get data with label 'Random Sample', get random samples from 2.
n_each_labels = len(EventsDataFrame_C02_C04)/2
n_random_samples = int(1.5*n_each_labels)

randomSamples = Dataset_C02_C04.sample(n=n_random_samples, random_state=111)
error=False

# 4. Check there are no repetead values
for i in randomSamples.index:
    for j in EventsDataFrame_C02_C04.index:
        if i==j:
            error=True

if error==True:
    print("ERROR:repeat random sampling")
else:
    print("Random Sampling OK")

Random Sampling OK


In [36]:
# 5. Make labels for the random samples

randomSamples["Label"] = np.ones([len(randomSamples),1],dtype=int)

randomSamples["Label"] = list( map( lambda val: 'Random Sample', list(range(len(randomSamples)))))
randomSamples=randomSamples.reset_index() 

indice = randomSamples.pop("Fecha")
randomSamples.index = indice
randomSamples


,Value (%),Value (hPa),Value (W/m2),Value (°C),Value (mm),Monthly sin,Monthly cos,Bi-monthly sin,Bi-monthly cos,Wind X,Wind Y,Label
Fecha,,,,,,,,,,,,
2019/12/21 17:55:00,91.0,663.3,14.0,9.8,0.0,0.595407,-0.803424,0.949585,0.313509,1.433948,1.246512,Random Sample
2019/12/19 08:50:00,95.0,665.4,618.0,10.4,0.0,0.906922,-0.421300,0.843000,0.537913,1.688381,2.599879,Random Sample
2018/04/06 23:40:00,92.0,665.8,0.0,4.9,0.0,-0.620235,-0.784416,-0.944568,0.328317,-0.566312,-0.411450,Random Sample
2018/07/01 22:55:00,77.0,665.7,0.0,5.1,0.0,0.174364,-0.984681,0.996163,0.087518,-0.435711,-0.670936,Random Sample
2019/03/24 18:30:00,98.0,665.8,0.0,9.0,0.0,0.868199,-0.496217,-0.864933,-0.501888,0.035837,0.093358,Random Sample
...,...,...,...,...,...,...,...,...,...,...,...,...
2019/05/16 17:40:00,76.0,664.9,12.0,9.9,0.0,0.578332,0.815801,-0.303479,-0.952838,-0.195366,-0.460252,Random Sample
2019/12/08 12:25:00,77.0,684.2,476.0,13.2,0.0,-0.263733,0.964596,-0.133049,0.991109,-1.034814,1.593474,Random Sample
2018/03/16 01:20:00,73.0,664.9,0.0,8.7,0.0,-0.705047,0.709161,0.381339,-0.924435,1.714335,1.030076,Random Sample


In [37]:
# 6. Concatenate Random Samples with Events DataFrame and make the final training dataset

Training_Dataset = pd.concat([EventsDataFrame_C02_C04, randomSamples])
Training_Dataset

,Value (%),Value (hPa),Value (W/m2),Value (°C),Value (mm),Monthly sin,Monthly cos,Bi-monthly sin,Bi-monthly cos,Wind X,Wind Y,Label
Fecha,,,,,,,,,,,,
2018/02/05 21:05:00,100.0,665.4,0.0,8.8,0.1,-0.6023956480982829,-0.798197646671706,-0.948208217289775,0.317649455633324,1.2944271909999199,0.940456403667957,Extreme event is near
2018/02/05 21:10:00,100.0,665.4,0.0,8.8,0.2,-0.602975954467736,-0.797759361169596,-0.9480926540084559,0.317994212864325,0.877624441943005,0.8183980320749978,Extreme event is near
2018/02/05 21:15:00,100.0,665.5,0.0,8.8,0.3,-0.603555941953558,-0.797320653772717,-0.9479769653775129,0.318338928052542,1.02308553935848,1.3576803670804,Extreme event is near
2018/02/05 21:20:00,100.0,665.5,0.0,8.8,0.1,-0.604135610249023,-0.7968815247130779,-0.947861151412241,0.318683601152398,1.1114533927344,1.15094368054184,Extreme event is near
2018/02/05 21:25:00,100.0,665.6,0.0,8.8,0.1,-0.604714959047574,-0.796441974222913,-0.9477452121279519,0.319028232118325,0.984088543485761,1.13206437033416,Extreme event is near
...,...,...,...,...,...,...,...,...,...,...,...,...
2019/05/16 17:40:00,76,664.9,12,9.9,0,0.578332,0.815801,-0.303479,-0.952838,-0.195366,-0.460252,Random Sample
2019/12/08 12:25:00,77,684.2,476,13.2,0,-0.263733,0.964596,-0.133049,0.991109,-1.03481,1.59347,Random Sample
2018/03/16 01:20:00,73,664.9,0,8.7,0,-0.705047,0.709161,0.381339,-0.924435,1.71433,1.03008,Random Sample


In [38]:
path = '/home/juan/Desktop/TESIS/Codes/codesTesis/Paso2/'
Training_Dataset.to_csv(path+'Training_Dataset.csv')
np.round(Training_Dataset.describe(),1).to_csv(path+'Training_Dataset_clasification_description.csv')


In [39]:
path = '/home/juan/Desktop/TESIS/Codes/codesTesis/Paso2/'
Training_Dataset = pd.read_csv(path+'Training_Dataset.csv', index_col=0)

np.round(Training_Dataset.describe(),1).to_csv(path + 'Training_Dataset_clasification_describe.csv')
np.round(Training_Dataset.describe(),1)

,Value (%),Value (hPa),Value (W/m2),Value (°C),Value (mm),Monthly sin,Monthly cos,Bi-monthly sin,Bi-monthly cos,Wind X,Wind Y
count,714.0,714.0,714.0,714.0,714.0,714.0,714.0,714.0,714.0,714.0,714.0
mean,86.1,671.6,94.0,9.4,0.5,0.1,-0.1,-0.1,-0.0,0.2,0.5
std,13.1,8.9,196.2,2.4,1.0,0.7,0.7,0.7,0.7,1.3,1.5
min,21.0,662.9,0.0,4.3,0.0,-1.0,-1.0,-1.0,-1.0,-5.0,-2.5
25%,76.2,664.9,0.0,7.5,0.0,-0.6,-0.8,-0.8,-0.7,-0.5,-0.6
50%,91.0,665.7,12.0,8.9,0.0,0.2,-0.1,-0.2,0.1,0.2,0.3
75%,97.0,682.8,76.8,11.2,0.4,0.9,0.4,0.7,0.6,1.0,1.1
max,100.0,687.4,1073.0,17.3,5.8,1.0,1.0,1.0,1.0,7.3,7.6
